In [1]:
from ebooklib import epub
import os
from multiprocessing import Pool
import time
from bs4 import BeautifulSoup
import urllib2
import string

In [ ]:
d="/media/some/sophie-hhd/books/"
ls=!find $d -type f -iname "*.html" -o -iname "*.htm" -o -iname "*.xhtml" -o -iname "*.xhtm" 
len(ls)
ls

In [ ]:
#remove html tags from string html     
def html2txt(html,ff):
    
    soup = BeautifulSoup(open(html).read(),"lxml")
    txt = soup.get_text()
    #print type(txt)
    m,f=os.path.split(html)
    open(ff,"w").write("".join(str(t) for t in txt if t in string.printable[:-3]))
    return ff

In [ ]:
#remove html tags from string html     
def txt(inp):
    d,f=os.path.split(inp)
    ff="/tmp/"+f    
    txt=open(inp).read()
    open(ff,"w").write("".join(str(t) for t in txt if t in string.printable[:-3]))
    return ff

In [ ]:
def mvht(l):
    ddir="/media/some/sophie-hhd/books/htm2epub/"
    m,f=os.path.split(l)
    if f[-1] == 'l':
        n=-4
    else:
        n=-3
    ff=ddir+f[:n]+"epub"
    fh=ddir+f[:n]+"txt"
    fe=m+"/"+f[:n]+"epub"
    tmp="/tmp/"+f[:n]+"epub"
    tmpl="/tmp/"+f[:n]+"txt"
    try:
        html2txt(l,tmpl)
        #os.remove(l)
    except:
        print "file doesnt exist",l
    !mv "$tmpl" "$fh"
    if not os.path.exists(ff) and not os.path.exists(fe):
        try:
            !ebook-convert "$tmpl" "$tmp" >/dev/null
            if os.path.exists(tmp):
                !mv "$tmp" "$ff"
        except:
            print "faulty   ",l
            
    print fh
 

In [ ]:
mvht(ls[0])

In [ ]:
t0 = time.time()
p = Pool(4)
p.map(mvht, ls)
p.terminate()
p.close()
t1 = time.time()
total = t1-t0
print('time: ',total)

In [2]:
#verify epubs
d="/media/some/sophie-hhd/books/thrill/"
lst=! find "$d" -type f -iname "*.epub"
len(lst)

6531

In [4]:
def verifye(f):
    m,ff=os.path.split(f)
    try:
        if os.path.exists(f):  
            book=epub.read_epub(f)
            os.remove(f)
            epub.write_epub(f,book)
            return (f,"OK")
        else:
            return (f,"DOESNT EXIST")
            #print "file  ..." ,ff," ...... verified"
    except:
        #print "file corrupt ..." ,ff," ...... deleting"
        #os.remove(f)
        return (f,"NOT OK")

In [5]:
print verifye(lst[0])


('/media/some/sophie-hhd/books/thrill/great hunt, The - Robert Jordan.epub', 'OK')


In [ ]:
t0 = time.time()
p = Pool(20)
m=map(verifye, lst)
p.terminate()
p.close()
t1 = time.time()
total = t1-t0
print('time: ',total)

/usr/local/lib/python2.7/dist-packages/ebooklib/epub.py:848: UserWarning: Duplicate name: 'EPUB/Images/cover0001.jpg'
  self.out.writestr('%s/%s' % (self.book.FOLDER_NAME, item.file_name), item.get_content())


In [176]:
cnt=0
for l in m:
    if l != None and l[1] =="NOT OK":
        cnt+=1
        #os.remove(l[0])
cnt

505

In [177]:
len(m)

33828